# Language Identification



##### Commet Configurations

### Libraries

In [ ]:
%timeit
!pip install nlppreprocess

In [ ]:
import re
import string

import pandas as pd
import numpy as np

# Dataset

In [ ]:
data = pd.read_csv('input/train_set.csv', encoding='UTF-8')
test = pd.read_csv('input/test_set.csv' ,encoding='UTF-8')

#Insurance Dataset
data_copy = data.copy()

In [ ]:
data.head()

,lang_id,text
0,xho,umgaqo-siseko wenza amalungiselelo kumaziko ax...
1,xho,i-dha iya kuba nobulumko bokubeka umsebenzi na...
2,eng,the province of kwazulu-natal department of tr...
3,nso,o netefatša gore o ba file dilo ka moka tše le...
4,ven,khomishini ya ndinganyiso ya mbeu yo ewa maana...


### Data Preprocessing

In [ ]:
data.lang_id.describe()

count     33000
unique       11
top         tso
freq       3000
Name: lang_id, dtype: object

In [ ]:
#Character Length
data_copy['length'] = data_copy['text'].apply(lambda x: len(x))

In [ ]:
data.describe()

,lang_id,text
count,33000,33000
unique,11,29948
top,tso,ngokwesekhtjheni yomthetho ophathelene nalokhu...
freq,3000,17


## Data Exploration (EDA)

In [ ]:
#Remove Punctuations
def _remove_punc(x):
    """
    Func removes punctuation and ASCII character
    using string.punctuation function
    
    Args:
        data: pandas dataframe
    Return:
        Dataframe: clean tweets
    """
    x = re.sub(r'[-]',' ',x)  # Replaces hyphens with a space
    x = re.sub(r'[_]', ' ', x)  # Replaces underscores with a space
    x = re.sub(r'[^\w\s]','',x)  # Removes non-word and non-space characters
    x = re.sub('[0-9]+', '', x)  # Removes numbers
    x = re.sub(r'[^\x00-\x7f]', r'', x)  # Removes non-ASCII characters
    return x

In [ ]:
def _lower(x):
    return x.lower()
data_copy['lower'] = data_copy['clean_punc'].apply(_lower)

In [ ]:
from nlppreprocess import NLP
nlp = NLP()
nlp.process('couldnt')

'could not'

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
#Remove Stopwords
nltk.download('stopwords')

# Load English stopwords
stopword = set(stopwords.words('english'))

def remove_stopwords(x):
    """
    Remove stop words in the dataset to reduce noise.
    
    Args:
        x: Text input (string)
    
    Return:
        Cleaned text without stopwords
    """
    # Tokenize the text
    words = x.split()
    
    # Remove stopwords
    words = [word for word in words if word.lower() not in stopword]
    
    # Join the words back into a string
    return ' '.join(words)

# Apply the function to the 'lower' column and create a new column 'Text_nonstop'
data_copy['Text_nonstop'] = data_copy['lower'].apply(remove_stopwords)

In [ ]:
def _analyzer(x):
    """
    Function combines all the cleaning operations
    
    Args:
        x: Text input (string)
    
    Return:
        Cleaned text
    """
    x = _remove_punc(x)
    x = _lower(x)
    x = remove_stopwords(x)
    return x

In [ ]:
data_copy.head()

,lang_id,text,length,clean_punc,lower,Text_nonstop
0,xho,umgaqo-siseko wenza amalungiselelo kumaziko ax...,220,umgaqo siseko wenza amalungiselelo kumaziko ax...,umgaqo siseko wenza amalungiselelo kumaziko ax...,umgaqo siseko wenza amalungiselelo kumaziko ax...
1,xho,i-dha iya kuba nobulumko bokubeka umsebenzi na...,252,i dha iya kuba nobulumko bokubeka umsebenzi na...,i dha iya kuba nobulumko bokubeka umsebenzi na...,dha iya kuba nobulumko bokubeka umsebenzi naph...
2,eng,the province of kwazulu-natal department of tr...,264,the province of kwazulu natal department of tr...,the province of kwazulu natal department of tr...,province kwazulu natal department transport in...
3,nso,o netefatša gore o ba file dilo ka moka tše le...,217,o netefata gore o ba file dilo ka moka te le d...,o netefata gore o ba file dilo ka moka te le d...,o netefata gore o ba file dilo ka moka te le d...
4,ven,khomishini ya ndinganyiso ya mbeu yo ewa maana...,239,khomishini ya ndinganyiso ya mbeu yo ewa maana...,khomishini ya ndinganyiso ya mbeu yo ewa maana...,khomishini ya ndinganyiso ya mbeu yo ewa maana...


In [ ]:
data['cleaned'] = data['text'].apply(_analyzer)
test['cleaned'] = test['text'].apply(_analyzer)

In [ ]:
# Splitting  X (indepedent) and Y (target/dependent) variables
X = data['cleaned']
y = data['lang_id']

In [ ]:
# from sklearn.preprocessing import LabelEncoder
# lab_enc = LabelEncoder()
# y = lab_enc.fit_transform(y)

### Train Test Split

In [ ]:
from sklearn.model_selection import train_test_split
X_train , X_test , y_train , y_test = train_test_split(X , y, stratify=y,
                                                       test_size =0.4, 
                                                       random_state=42)

In [ ]:
# Models
from sklearn.svm import LinearSVC, SVC
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression, SGDClassifier, RidgeClassifier
from sklearn.model_selection import cross_val_score, GridSearchCV, RepeatedStratifiedKFold
from sklearn.ensemble import RandomForestClassifier


In [ ]:
alg = [LogisticRegression(random_state =42 , max_iter=5000) , 
       MultinomialNB(), LinearSVC(random_state=42), 
       SGDClassifier(random_state=42), RidgeClassifier(random_state=42)]

In [ ]:
from sklearn import metrics
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer , TfidfVectorizer

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import metrics

def _performance_assessment(X_train, X_test, y_train, y_test, alg):
    model_stats = {}

    for clf in alg:
        model = Pipeline([
            ('tfidf', TfidfVectorizer(stop_words='english', max_df=0.9, ngram_range=(1, 5), analyzer='char')),
            ('clf', clf)
        ])
    
        model.fit(X_train, y_train)  # Training
        model_pred = model.predict(X_test)  # Testing

        # Dictionary of Models Performances
        model_stats[clf.__class__.__name__] = {
            'F1-Macro': metrics.f1_score(y_test, model_pred, average='macro'),
            'F1-Micro': metrics.f1_score(y_test, model_pred, average='micro'),
            'F1-Weighted': metrics.f1_score(y_test, model_pred, average='weighted')
        }

    return pd.DataFrame.from_dict(model_stats, orient='index')


In [ ]:
performance = _performace_assesment(alg , X_train , X_test , y_train , y_test)
performance.to_csv('performance.csv')
dataframe = pd.read_csv('performance.csv', index_col = 0)
dataframe.sort_values('F1-Weighted', ascending=False)

,F1-Macro,F1-Accuracy,F1-Weighted
MultinomialNB,0.999394,0.999394,0.999394
RidgeClassifier,0.999167,0.999167,0.999167
LinearSVC,0.999167,0.999167,0.999167
SGDClassifier,0.999091,0.999091,0.999091
LogisticRegression,0.998183,0.998182,0.998183


In [ ]:
  best_params = []

    for clf in alg:
        model = Pipeline([
            ('tfidf', TfidfVectorizer(stop_words='english', max_df=0.9, ngram_range=(1, 5), analyzer='char')),
            ('clf', clf)
        ])
        model.fit(X_train, y_train)  # Training
        
        # Get models performing parameters
        params = model.get_params()
        model_params = {'model_name': clf.__class__.__name__}

        for key in params:
            if key.startswith("clf"):
                if len(key) < 5:
                    model_params['model'] = params[key]
                else:
                    model_params[key[5:]] = params[key]

        best_params.append(model_params)

    return best_params

In [ ]:
best_params = _param_tuning(alg, X_train, y_train)

In [ ]:
#Best parameters
best_params

{'LinearSVC': {'C': 1.0,
  'class_weight': None,
  'dual': True,
  'fit_intercept': True,
  'intercept_scaling': 1,
  'loss': 'squared_hinge',
  'max_iter': 1000,
  'model': LinearSVC(random_state=42),
  'multi_class': 'ovr',
  'penalty': 'l2',
  'random_state': 42,
  'tol': 0.0001,
  'verbose': 0},
 'LogisticRegression': {'C': 1.0,
  'class_weight': None,
  'dual': False,
  'fit_intercept': True,
  'intercept_scaling': 1,
  'l1_ratio': None,
  'max_iter': 5000,
  'model': LogisticRegression(max_iter=5000, random_state=42),
  'multi_class': 'auto',
  'n_jobs': None,
  'penalty': 'l2',
  'random_state': 42,
  'solver': 'lbfgs',
  'tol': 0.0001,
  'verbose': 0,
  'warm_start': False},
 'MultinomialNB': {'alpha': 1.0,
  'class_prior': None,
  'fit_prior': True,
  'model': MultinomialNB()},
 'RidgeClassifier': {'alpha': 1.0,
  'class_weight': None,
  'copy_X': True,
  'fit_intercept': True,
  'max_iter': None,
  'model': RidgeClassifier(random_state=42),
  'normalize': 'deprecated',
  'pos

### GridSearchCV

#### Applying MultinomialNB()

In [ ]:
#model
model1 = MultinomialNB()

In [ ]:
Vectorize = TfidfVectorizer(stop_words = 'english', max_df=0.9, ngram_range=(1, 5), analyzer= 'char')
X_train = Vectorize.fit_transform(X_train)
X_test = Vectorize.transform(X_test)

In [ ]:
stratified_kfold = StratifiedKFold(n_splits=5, shuffle=True,
                                   random_state=42)

In [ ]:
best_params[alg[1].__class__.__name__]

{'alpha': 1.0,
 'class_prior': None,
 'fit_prior': True,
 'model': MultinomialNB()}

In [ ]:
alpha = list(np.linspace(0.1,0.02,4))
param_grid = dict(alpha=alpha)
grid_search = GridSearchCV(estimator= model1,
                           param_grid=param_grid,
                           scoring='f1_weighted',
                           cv=stratified_kfold,
                           error_score=0,
                           n_jobs=-1)

In [ ]:
grid_search.fit(X_train, y_train)
prediction = grid_search.predict(X_test)
cv_score = grid_search.best_score_
test_score = grid_search.score(X_test, y_test)

In [ ]:
print(f'Cross-validation score: {cv_score}')
print(f'Test score: {test_score}')
grid_search.best_params_    
grid_search.best_estimator_

Cross-validation score: 0.9996969686252897
Test score: 0.9995455174533855


MultinomialNB(alpha=0.04666666666666667)

#### Applying RidgeRegression()

In [ ]:
#model
model2 = RidgeClassifier()

In [ ]:
best_params[alg[4].__class__.__name__]

{'alpha': 1.0,
 'class_weight': None,
 'copy_X': True,
 'fit_intercept': True,
 'max_iter': None,
 'model': RidgeClassifier(random_state=42),
 'normalize': 'deprecated',
 'positive': False,
 'random_state': 42,
 'solver': 'auto',
 'tol': 0.001}

In [ ]:
alpha = list(np.linspace(0.15,0.4, 5))
param_grid = dict(alpha=alpha)
grid_search = GridSearchCV(estimator= model2,
                           param_grid=param_grid,
                           scoring='f1_weighted',
                           cv=stratified_kfold,
                           error_score=0,
                           n_jobs=-1)

In [ ]:
grid_search.fit(X_train, y_train)
prediction = grid_search.predict(X_test)
cv_score = grid_search.best_score_
test_score = grid_search.score(X_test, y_test)

In [ ]:
print(f'Cross-validation score: {cv_score}')
print(f'Test score: {test_score}')
grid_search.best_params_    
grid_search.best_estimator_

Cross-validation score: 0.9994949477404799
Test score: 0.9991666977456909


RidgeClassifier(alpha=0.2125)

In [ ]:
from sklearn.ensemble import StackingClassifier
from sklearn.pipeline import make_pipeline


In [ ]:
# X = data['cleaned']
# y = data['lang_id']

# lab_enc = LabelEncoder()
# y = lab_enc.fit_transform(y)

In [ ]:
X_train , X_test , y_train , y_test = train_test_split(X, y,  stratify=y, test_size=0.4, random_state =1)

In [ ]:
vect = TfidfVectorizer(stop_words = 'english', max_df=0.9, ngram_range=(2, 6), analyzer= 'char')
X_train = vect.fit_transform(X_train)
X_test = vect.transform(X_test)

In [ ]:
multiNB1 = MultinomialNB(alpha=0.1)
multiNB2 = MultinomialNB(alpha=0.1)

estimators = [('multiNB1', multiNB1), ('multiNB2', multiNB2)]
final_est = RidgeClassifier(alpha=0.2125)

In [ ]:
stacking_NB2 = StackingClassifier(estimators = estimators,
                           final_estimator = final_est,
                           passthrough = True)


In [ ]:
stacking_NB2.fit(X_train , y_train)

StackingClassifier(estimators=[('multiNB1', MultinomialNB(alpha=0.1)),
                               ('multiNB2', MultinomialNB(alpha=0.1))],
                   final_estimator=RidgeClassifier(alpha=0.2125),
                   passthrough=True)

In [ ]:
pred = stacking_NB2.predict(X_test)


In [ ]:
model_stats = {}
model_stats[stacking_NB2.__class__.__name__] = {
        'F1-Macro':metrics.f1_score(y_test, pred, average='macro'),
        'F1-Accuracy':metrics.f1_score(y_test, pred, average='micro'),
        'F1-Weighted':metrics.f1_score(y_test, pred, average='weighted')}
pd.DataFrame.from_dict(model_stats, orient='index')

,F1-Macro,F1-Accuracy,F1-Weighted
StackingClassifier,0.999773,0.999773,0.999773


In [ ]:
count_vec = CountVectorizer(ngram_range=(3,7), analyzer= 'char')
X_train , X_test , y_train , y_test = train_test_split(X, y, stratify=y,test_size=0.05, random_state =1)
X_train = count_vec.fit_transform(X_train)
X_test = count_vec.transform(X_test)

In [ ]:
multiNB1 = MultinomialNB(alpha=0.1)
multiNB2 = MultinomialNB(alpha=0.1)
multiNB3 = MultinomialNB(alpha=0.1)

estimators = [('multiNB1', multiNB1), ('multiNB2', multiNB2), ('multiNB3', multiNB3)]
final_est = RidgeClassifier(alpha=0.2125)

In [ ]:
stacking_NB3 = StackingClassifier(estimators = estimators,
                           final_estimator = final_est,
                           passthrough = True)

In [ ]:
stacking_NB3.fit(X_train , y_train)

In [ ]:
pred = stacking_NB3.predict(X_test)

KeyboardInterrupt: ignored

In [ ]:
model_stats = {}
model_name = stacking_NB3.__class__.__name__
pred = stacking_NB3.predict(X_test)

metrics_dict = {
    'F1-Macro': 'macro',
    'F1-Micro': 'micro',
    'F1-Weighted': 'weighted'
}

model_scores = {}
for metric_name, average_method in metrics_dict.items():
    score = metrics.f1_score(y_test, pred, average=average_method)
    model_scores[metric_name] = score

model_stats[model_name] = model_scores
df_model_stats = pd.DataFrame.from_dict(model_stats, orient='index')

print(df_model_stats)

,F1-Macro,F1-Accuracy,F1-Weighted
StackingClassifier,1.0,1.0,1.0


### Kaggle Submission

In [ ]:
X = test['cleaned']
Vectorize = vect.transform(X)

In [ ]:
test['lang_id'] = stacking_NB2.predict(Vectorize)

In [ ]:
submission = test[['index', 'lang_id']]
submission.to_csv('Submission.csv',index=False)
submission

,index,lang_id
0,1,6
1,2,2
2,3,8
3,4,5
4,5,0
...,...,...
5677,5678,1
5678,5679,3
5679,5680,4
5680,5681,4


In [ ]:
test.cleaned

0       mmasepala fa maemo kgethegileng letlelela kgat...
1       uzakwaziswa ngokufaneleko nakungafuneka eminye...
2               tshivhumbeo tshi fana na ngano dza vhathu
3       kube inja nelikati betingevakala kutsi titsini...
4                            winste op buitelandse valuta
                              ...                        
5677                      you mark your ballot in private
5678    ge o ka kgetha ka bowena go se omie mofani ka ...
5679    e ka kopo etsa kgetho ya hao ka hloko hobane h...
5680    tb ke bokudi ba pmb mme morero o tla lefella t...
5681                 vakatjhela iwebhusayidi yethu ku www
Name: cleaned, Length: 5682, dtype: object